In [ ]:
import pandas as pd

Proszę utworzyć dataframe   z https://covid19.who.int/WHO-COVID-19-global-data.csv  a następnie złączyć je z danymi z pliku https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv w celu uzyskania informacji o kontynencie na którym leży kraj. Tu mała podpucha jest, uwaga :)

    Pogrupować dane według dnia i kontynentu, aby znaleźć sumaryczną liczbę zachorowań i zgonów (wizualizacja w załączniku covid1.png); zaprezentować te dane w postaci wykresu liniowego.
    Obliczyć średnią  14-dniową zachorowań dla Polski oraz Niemiec (wizualizacja w załączniku covid2.png) i zaprezentować w formie wykresu

In [ ]:
url_covid = "https://covid19.who.int/WHO-COVID-19-global-data.csv"
url_continents = "https://pkgstore.datahub.io/JohnSnowLabs/country-and-continent-codes-list/country-and-continent-codes-list-csv_csv/data/b7876b7f496677669644f3d1069d3121/country-and-continent-codes-list-csv_csv.csv"

In [ ]:
df_covid = pd.read_csv(url_covid)
df_covid

In [ ]:
df_covid.describe()

In [ ]:
# usuwanie wartosci <0
df_covid = df_covid.query(" New_cases>=0 and New_deaths>=0 ")

In [ ]:
df_continents = pd.read_csv(url_continents)
df_continents

In [ ]:
len(df_continents), len(df_continents.Two_Letter_Country_Code.unique())

In [ ]:
df_continents[ df_continents.duplicated(subset="Two_Letter_Country_Code") ].sort_values('Two_Letter_Country_Code')

In [ ]:
df_continents.query(" Two_Letter_Country_Code=='TR' ")

In [ ]:
# usuwanie duplikatow z df_continents
df_continents.drop_duplicates(subset="Two_Letter_Country_Code", inplace=True)

In [ ]:
len(df_continents), len(df_continents.Two_Letter_Country_Code.unique())

In [ ]:
df_covid.query(" Country == 'Kosovo[1]' ")

In [ ]:
df_continents.query(" Two_Letter_Country_Code=='XK' ")

In [ ]:
# merge danych
df = pd.merge(df_covid, df_continents, how="left", 
         left_on="Country_code", right_on="Two_Letter_Country_Code")
df.sample(n=10, random_state=0)

In [ ]:
# grupowanie na podstawie daty i kontynentu
dfg = df.groupby(["Date_reported","Continent_Name"])
df1 = dfg["New_cases","New_deaths"].sum()
df1

In [ ]:
df1.reset_index(inplace=True)
df1.columns = ["Date","Continent","Cases","Deaths"]
df1.set_index("Date", inplace=True)
df1

In [ ]:
df1.query("Continent=='Europe'")["Deaths"].plot(figsize=(21,7))

In [ ]:
# pobieranie danych tylko dla PL i DE
df2 = df_covid.query(" Country_code=='PL' or Country_code=='DE' ")
df2["Date_reported"] = pd.to_datetime(df2.Date_reported)
df2.set_index("Date_reported", inplace=True)
df2

In [ ]:
df2.index

In [ ]:
covid_PL = df2.query(" Country_code=='PL' ").New_cases.resample("14D").mean()
covid_PL

In [ ]:
covid_DE = df2.query(" Country_code=='DE' ").New_cases.resample("14D").mean()
covid_DE

In [ ]:
df_14D = pd.concat([covid_PL, covid_DE], axis=1)
df_14D.columns = ['Polska','Niemcy']
df_14D

In [ ]:
df_14D.plot(figsize=(21,8), grid=True, kind='bar')